In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score

# suppress warnings
import warnings
warnings.filterwarnings('ignore')


In [67]:
# read the data
df = pd.read_csv('https://raw.githubusercontent.com/nikjohn7/Disaster-Tweets-Kaggle/main/data/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [68]:
# how many rows and columns are in the data set?
df.shape

(7613, 5)

In [69]:
# how many tweets are there that contain keyword "battle"?
df[df['keyword'] == 'battle'].shape

(26, 5)

In [70]:
# how many tweets are there that contain keyword "avalanche"?
df[df['keyword'] == 'avalanche'].shape

(30, 5)

In [71]:
import nltk
nltk.download('stopwords')

stopwords = set(nltk.corpus.stopwords.words('english'))

# include_stopwords = {'dear', 'regards', 'must', 'would', 'also', 
#                      'canada', 'canadian', 'canadians', 'prime', 'minister', 'province', 'provinces'
#                      'united', 'states', 'state', 'america', 'american', 'president','secretary'}
include_stopwords = {'ablaze', 'aftershock', 'battle', 'army', 'ambulance', 
                     'annihilation', 'armageddon', 'apocalypse', 'arson', 'arsonist', 'attacked', 'avalanche'
                     'bioterrorism', 'blazing', 'bleeding', 'blight', 'bomb', 'casualties','catastrophic', 'injuries'}
exclude_stopwords = {'against'}

stopwords |= include_stopwords
stopwords -= exclude_stopwords

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
# build a text processing and classifier pipeline
# to predict the target (Disaster or Non-Disaster) of a tweet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

df2 = df[df['target'].isin([0, 1])] # 0 is not a disaster and 1 is a disaster
df2.head()

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df2['text'], df2['target'], test_size=0.2)

# Create a pipeline that first transforms the text data into TF-IDF vectors, then applies SVM
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=list(stopwords))),
    ('clf', svm.SVC()),
])

# Train the classifier
text_clf.fit(X_train, y_train)

# Predict the test set results
y_pred = text_clf.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred, target_names=['Disaster', 'Non-Disaster']))


              precision    recall  f1-score   support

    Disaster       0.78      0.89      0.83       878
Non-Disaster       0.81      0.67      0.73       645

    accuracy                           0.79      1523
   macro avg       0.80      0.78      0.78      1523
weighted avg       0.80      0.79      0.79      1523



In [73]:
# This script creates a new column 'sentiment' in the dataframe, 
# which contains the sentiment score of the text. 
# The sentiment score is a float within the range [-1.0, 1.0], 
# where -1.0 denotes a very negative sentiment, 
# 1.0 denotes a very positive sentiment, 
# and values around 0 denote a neutral sentiment.

from textblob import TextBlob

# Define a function to apply sentiment analysis to a tweet text
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity  # returns a value between -1 and 1

# Create a new column 'sentiment' in the DataFrame
df2['sentiment'] = df2['text'].apply(get_sentiment)

print("FINISHED")

# Display the DataFrame
# df2

FINISHED


In [74]:
# find average sentiment for each target in df2
print("0: Non-Disaster and 1: Disaster")
df2.groupby('target')['sentiment'].mean()

0: Non-Disaster and 1: Disaster


target
0    0.070622
1    0.018631
Name: sentiment, dtype: float64

In [75]:
# find average sentiment for each keyword in df2
# order the results from most positive to most negative

df2.groupby('keyword')['sentiment'].mean().sort_values(ascending=False).head(5)

keyword
hazardous    0.457891
razed        0.418946
outbreak     0.312661
mayhem       0.277262
wreckage     0.273440
Name: sentiment, dtype: float64